In [1]:
!pip install nltk
!pip install numpy

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 465.6 kB/s eta 0:00:03
   ------------- -------------------------- 0.5/1.5 MB 465.6 kB/s eta 0:00:03
   -------------------- ------------------- 0.8/1.5 MB 500.8 kB/s eta 0:00:02
   -------------------- ------------------- 0.8/1.5 MB 500.8 kB/s eta 0:00:02
   --------------------------- ------------ 1.0/1.5 MB 498.4 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 498.4 kB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 4

In [2]:
!pip install keras tensorflow

  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached MarkupSafe-3.0.2-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.3 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.3 MB 492.8 kB/s eta 0:00:02
   --------------- ------------------------ 0.5/1.3 MB 492.8 kB/s eta 0:00:02
   --------------- ------------------------ 0.5/1.3 MB 492.8 kB/s eta 0:00:02
   ----------------------- ---------------- 0.8/1.3 MB 459.5 kB/s eta 0:00:02
   ----------------------- ---------------- 0.8/1.3 MB 459.5 kB/s eta 0:00:02
   ------------------------------- -------- 1.0/1.3 MB 493.4 kB/s eta 0:00:01
   ------------------------------- -------- 1.

In [ ]:
import json
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import tensorflow as tf

Reading Data :

In [ ]:
data_file = open('data.json').read()
intents = json.loads(data_file)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?' , '!', '.', ',', "'s", "'m", "'re", "'ll", "'ve", "'d", "'t"]